In [25]:
import numpy as np
from pprint import pprint

In [31]:
from scipy.signal import cont2discrete, lti, dlti, dstep
m = 5
dt = 0.01
A = np.vstack((np.hstack((np.zeros((3, 3)), np.eye(3))), np.zeros((3, 6))))
B = np.vstack((np.zeros((3, 3)), np.eye(3)/m))
C = np.array([[1., 0., 0.]])
D = np.array([[0.]])
M = cont2discrete((A, B, C, D), dt, method='zoh')
pprint(M)

(array([[1.  , 0.  , 0.  , 0.01, 0.  , 0.  ],
       [0.  , 1.  , 0.  , 0.  , 0.01, 0.  ],
       [0.  , 0.  , 1.  , 0.  , 0.  , 0.01],
       [0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 1.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 1.  ]]),
 array([[1.e-05, 0.e+00, 0.e+00],
       [0.e+00, 1.e-05, 0.e+00],
       [0.e+00, 0.e+00, 1.e-05],
       [2.e-03, 0.e+00, 0.e+00],
       [0.e+00, 2.e-03, 0.e+00],
       [0.e+00, 0.e+00, 2.e-03]]),
 array([[1., 0., 0.]]),
 array([[0.]]),
 0.01)


*That's all very well and good, but this is a ZOH approximation... Also, what if we want to solve this symbolically?*

In [70]:
import sympy as sp
import numpy.linalg

sp.var('t')
m = 5  # m must be non-symbolic for sympy to get eigs...

A = np.vstack((np.hstack((np.zeros((3, 3)), np.eye(3))), np.zeros((3, 6))))
B = np.vstack((np.zeros((3, 3)), np.eye(3)/m))
print("A = ")
pprint(A)
print("eigs of A = ")
pprint(np.linalg.eig(A)[0])
# pprint(np.linalg.inv(A))

A = 
array([[0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])
eigs of A = 
array([0., 0., 0., 0., 0., 0.])


In [71]:
AB = np.vstack((np.hstack((A, B)), np.zeros((3, 9))))
pprint(AB)
print(np.shape(AB))
print("eigs of AB = ")
pprint(np.linalg.eig(AB)[0])

array([[0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0.2, 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.2, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.2],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ]])
(9, 9)
eigs of AB = 
array([0., 0., 0., 0., 0., 0., 0., 0., 0.])


In [80]:
# redo with m as symbolic
%reset
import numpy as np
from pprint import pprint
import sympy as sp
import numpy.linalg

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


*Use eigenvalues to solve for e^At...*

In [81]:
sp.var('m, t')

A = np.vstack((np.hstack((np.zeros((3, 3)), np.eye(3))), np.zeros((3, 6))))
B = np.vstack((np.zeros((3, 3)), np.eye(3)/m))
AB = np.vstack((np.hstack((A, B)), np.zeros((3, 9))))
M = AB@AB*(1 + t**2)/2 + AB*t + np.eye(np.shape(AB)[0])
pprint(M)

array([[1.00000000000000, 0, 0, 1.0*t, 0, 0, 0.5*(t**2 + 1)/m, 0, 0],
       [0, 1.00000000000000, 0, 0, 1.0*t, 0, 0, 0.5*(t**2 + 1)/m, 0],
       [0, 0, 1.00000000000000, 0, 0, 1.0*t, 0, 0, 0.5*(t**2 + 1)/m],
       [0, 0, 0, 1.00000000000000, 0, 0, 1.0*t/m, 0, 0],
       [0, 0, 0, 0, 1.00000000000000, 0, 0, 1.0*t/m, 0],
       [0, 0, 0, 0, 0, 1.00000000000000, 0, 0, 1.0*t/m],
       [0, 0, 0, 0, 0, 0, 1.00000000000000, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1.00000000000000, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1.00000000000000]], dtype=object)


In [82]:
Ad = M[0:6, 0:6]
pprint(Ad)

array([[1.00000000000000, 0, 0, 1.0*t, 0, 0],
       [0, 1.00000000000000, 0, 0, 1.0*t, 0],
       [0, 0, 1.00000000000000, 0, 0, 1.0*t],
       [0, 0, 0, 1.00000000000000, 0, 0],
       [0, 0, 0, 0, 1.00000000000000, 0],
       [0, 0, 0, 0, 0, 1.00000000000000]], dtype=object)


In [83]:
Bd = M[0:6, 6:9]
pprint(Bd)

array([[0.5*(t**2 + 1)/m, 0, 0],
       [0, 0.5*(t**2 + 1)/m, 0],
       [0, 0, 0.5*(t**2 + 1)/m],
       [1.0*t/m, 0, 0],
       [0, 1.0*t/m, 0],
       [0, 0, 1.0*t/m]], dtype=object)


In [87]:
sp.var('x, y, z, dx, dy, dz, fx, fy, fz, g')
X = np.array([x, y, z, dx, dy, dz]).T
U = np.array([fx, fy, fz]).T
G = np.array([0, 0, 0, 0, 0, -g]).T
xk1 = Ad@X + Bd@U + G
pprint(xk1)

array([1.0*dx*t + 0.5*fx*(t**2 + 1)/m + 1.0*x,
       1.0*dy*t + 0.5*fy*(t**2 + 1)/m + 1.0*y,
       1.0*dz*t + 0.5*fz*(t**2 + 1)/m + 1.0*z, 1.0*dx + 1.0*fx*t/m,
       1.0*dy + 1.0*fy*t/m, 1.0*dz + 1.0*fz*t/m - g], dtype=object)
